# Full revision analysis

## List of work to be performed:
1. Supplementary figure showing incorrect inform tumor segmentation and NKT cells 
2. Supplementary figure with zoom-in images of individual cells from inForm and TME-Analyzer in an image with high disagreement 
3. Incorporation of DeepCell into TME-Analyzer 
4. Reorganize the MonteCarlo trials 
5. Supplementary table 6 with full ranking of parameters and discovery/validation p values if top n parameters were used instead. 
6. Generate forest plot for 4E 
7. Build classifier based on train-test split of discovery cohort. Try 16-47, 32-31, 47-16 splits. 
8. ? It is significant amount of work, but since both reviewers mention is, I can train a random forest on the original dataset and report it ? 

## 4. Reorganize the MonteCarlo trials 
We now provide a zip folder consisting of a 

4.1 .csv file listing the classifier parameters, 

4.2 a .txt file listing the patient split to test-train groups and 

4.3 the classifier performance, and 

4.4 a .png file plotting the Kaplan-Meier curves of patient groups; per classifier (a total of 3000 files, 70Mb).

task 4.2 & 4.3:

In [2]:
import pandas as pd
import numpy as np
import os
import shutil

Classifier_folder = "E:/Desktop_backup_20231031/Desktop/TME-Analyzer_the_manuscript/all_ML_approaches/v2/trials_20201202/first_1000/"
Classifier_folder = 'E:/Moved from E/TME-Analyzer_the_manuscript/20211129_TME-Analyzer/ML/individual classifiers/'
output_folder = "E:/TME-Analyzer-large-files-from-desktop-folder/20231207_revision_files/Task4/"
patient_lookup = pd.read_excel("E:/Moved from E/TME-Analyzer_the_manuscript/20211129_TME-Analyzer/data_analysis/reproduction_of_old_analysis/20230712/discovery_data.xlsx", engine='openpyxl')
patient_lookup = patient_lookup.set_index(patient_lookup.keys()[0]).iloc[:,0]
# print(patient_lookup)
px_size = 0.4999
classifier_files = np.sort([i for i in os.listdir(Classifier_folder) if (i[-4:] == '.txt') & ('classifier_2022' in i)])
n_i = 0
for n_i in range(1000):
    file_name = Classifier_folder + classifier_files[n_i]
    with open(file_name,'r') as f:
        lines = f.readlines()
    with open(output_folder + 'classifier_' + f'{n_i+1:04d}' + '.txt','w') as f:
        for line in lines:
            for i in patient_lookup.index:
                # print(i)
                # print(line,type(line))
                if str(i) in line:
                    line = line.replace(str(i),str(patient_lookup.loc[i]))
            # print(line)
            f.write(line)
        # print(lines)

task 4.1

In [5]:
for n_i in range(1000):
# n_i = 0
    file_name = Classifier_folder + classifier_files[n_i].replace('.txt','.xls')

    px_size_dora = 0.4999
    classifier_frame = pd.read_csv(file_name,sep='\t')

    classifier_frame_new = pd.DataFrame(classifier_frame[['mean (good prognosis)', 'mean (bad prognosis)','p-value','used for classifier?']])
    classifier_frame_new.index = classifier_frame['Unnamed: 0']#,columns = ['mean (good prognosis)', 'mean (bad prognosis)'])
    px_size_cols = ['mean (good prognosis)', 'mean (bad prognosis)']
    for i in classifier_frame_new.index:
        if 'area' in i:
            classifier_frame_new.loc[i,px_size_cols] = classifier_frame_new.loc[i,px_size_cols]*px_size_dora*px_size_dora
        elif 'density' in i:
            classifier_frame_new.loc[i,px_size_cols] = classifier_frame_new.loc[i,px_size_cols]/px_size_dora/px_size_dora

    new_index = []
    for i in classifier_frame_new.index:
        if 'found' in i:
            new_index.append(i[:i.find('found')-1])
        else:
            new_index.append(i[:i[::-1].find('ni')*-1-3])
    classifier_frame_new.index = new_index

    classifier_frame_new.to_excel(output_folder + 'classifier_' + f'{n_i+1:04d}' + '.xls')

task 4.4

In [6]:
for n_i in range(1000):
    shutil.copy(Classifier_folder + classifier_files[n_i].replace('.txt','_os.png'), output_folder + 'classifier_' + f'{n_i+1:04d}' + '.png')